In [8]:
import requests
import pandas as pd
import json
import re
import numpy as np
import asyncio
import os
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import time
from datetime import datetime
from IPython.display import clear_output


In [ ]:
TICKERS_CSV = "data/top100/tickers.csv"  
OUTPUT_DIR = "output/top100"                         
MAX_TICKERS = 100

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [10]:

df_tickers = pd.read_csv(TICKERS_CSV)

tickers = (
    df_tickers["symbol"]
    .dropna()
    .astype(str)
    .unique()
    .tolist()
)[:MAX_TICKERS]

print(f"Loaded {len(tickers)} tickers")
tickers[:10]


Loaded 100 tickers


['ARKK', 'ALV', 'TLT', 'ENVA', 'ASND', 'BELFB', 'PKE', 'ARWR', 'EVCM', 'CAE']

In [11]:
data = yf.download(
    tickers=" ".join(tickers),
    period="1d",
    interval="5m",
    group_by="ticker",
    threads=True,
    progress=False
)


C:\Users\a73s\AppData\Local\Temp\ipykernel_14520\1014172803.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(


In [12]:
records = []
timestamp = datetime.utcnow().isoformat()

for symbol in tickers:
    try:
        last_price = data[symbol]["Close"].dropna().iloc[-1]
        records.append({
            "symbol": symbol,
            "price": float(last_price),
            "timestamp": timestamp
        })
    except Exception:
        pass

prices_df = pd.DataFrame(records)
prices_df.head()


C:\Users\a73s\AppData\Local\Temp\ipykernel_14520\2810629483.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()


,symbol,price,timestamp
0,ARKK,80.294998,2025-12-20T20:58:54.943976
1,ALV,121.080002,2025-12-20T20:58:54.943976
2,TLT,87.480003,2025-12-20T20:58:54.943976
3,ENVA,163.639999,2025-12-20T20:58:54.943976
4,ASND,199.990005,2025-12-20T20:58:54.943976


In [13]:
csv_path = os.path.join(OUTPUT_DIR, "prices.csv")
prices_df.to_csv(csv_path, index=False)

print(f"Saved {csv_path}")

Saved output/top100\prices.csv


In [ ]:
html = "<!DOCTYPE html>\n<html>\n<head>\n<meta charset='UTF-8'>\n<title>Market Dashboard</title>\n"
# Link to external CSS
html += "<link rel='stylesheet' href='https://hotstockmarket.web.app/styles.css'>\n"
html += "</head>\n<body>\n"

# Table
html += "<table class='table-container' border='1' cellpadding='5' cellspacing='0'>\n<thead><tr><th>Symbol</th><th>Price</th><th>Timestamp</th></tr></thead>\n<tbody>\n"
for _, row in prices_df.iterrows():
    html += f"<tr><td>{row['symbol']}</td><td>{row['price']:.2f}</td><td>{row['timestamp']}</td></tr>\n"
html += "</tbody>\n</table>\n"

# End HTML
html += "</body>\n</html>"

# Save HTML
HTML = os.path.join(OUTPUT_DIR, "dashboard.html")
with open(HTML, "w", encoding="utf-8") as f:
    f.write(html)

print("HTML file generated: dashboard.html")


HTML file generated: dashboard.html


In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import os

for symbol in tickers:
    try:
        series = data[symbol]["Close"].dropna()

        if series.empty:
            continue

        last_price = series.iloc[-1]
        last_time = series.index[-1]

        plt.figure(figsize=(6, 4))
        plt.plot(series.index, series.values, linewidth=1)
        plt.grid(True)

        # Title
        plt.title(f"{symbol}", fontsize=12)

        # Last price marker
        plt.scatter(last_time, last_price, zorder=3)

        # Last price label
        plt.text(
            last_time,
            last_price,
            f" {last_price:.2f}",
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="left",
            bbox=dict(facecolor="white", edgecolor="none", alpha=0.8)
        )

        plt.tight_layout()

        chart_path = os.path.join(OUTPUT_DIR, f"{symbol}.png")
        plt.savefig(chart_path, dpi=150)
        plt.close()

        print(f"Saved {chart_path}")

    except Exception as e:
        print(f"Skipping {symbol}: {e}")


Saved output/top100/charts\ARKK.png
Saved output/top100/charts\ALV.png
Saved output/top100/charts\TLT.png
Saved output/top100/charts\ENVA.png
Saved output/top100/charts\ASND.png
Saved output/top100/charts\BELFB.png
Saved output/top100/charts\PKE.png
Saved output/top100/charts\ARWR.png
Saved output/top100/charts\EVCM.png
Saved output/top100/charts\CAE.png
Skipping OCUL	: 'OCUL\t'
Saved output/top100/charts\PRAX.png
Saved output/top100/charts\NRIX.png
Saved output/top100/charts\STOK.png
Saved output/top100/charts\BTG.png
Skipping PATH    : 'PATH    '
Skipping CIEN	: 'CIEN\t'
Skipping CENX	: 'CENX\t'
Skipping LMND	: 'LMND\t'
Skipping PL 		: 'PL \t\t'
Saved output/top100/charts\IMMX.png
Saved output/top100/charts\AFRM.png
Saved output/top100/charts\BLTE.png
Saved output/top100/charts\GILD.png
Saved output/top100/charts\TSEM.png
Saved output/top100/charts\CVNA.png
Saved output/top100/charts\WVE.png
Saved output/top100/charts\GPCR.png
Saved output/top100/charts\KYMR.png
Saved output/top100/c